To run this file & observe the outputs of our model and the results of the bert & rogue evlauation metrics simply "Run All" and make sure you are connected to a GPU (you may have to create a shortcut from the shared folder to "My Drive") prior to these steps

In [1]:
# Install necessary huggingface packages
#%%capture
!pip install -U transformers datasets
!pip install --upgrade accelerate
!pip install evaluate bert_score rouge_score
!pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 22.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 2.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=3d64c42ce2b9ab8893462f05eaeb9c5ee00812a69765d1fbcfd9e69528465460
  Stored in directory: /root/.cache/pip/wheels/5f/dd

In [2]:
# Import packages
from transformers import pipeline, Trainer, TrainingArguments, DataCollatorForLanguageModeling, AutoTokenizer, AutoModelForSeq2SeqLM, TFAutoModelForSeq2SeqLM, Seq2SeqTrainingArguments, Seq2SeqTrainer, T5ForConditionalGeneration
from evaluate import load
from datasets import Dataset
from transformers import DataCollatorForSeq2Seq
import numpy as np
import pandas as pd
import os
import torch
import copy
from transformers import T5ForConditionalGeneration, T5Tokenizer
import sentencepiece
from transformers import AutoTokenizer

from collections import defaultdict

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device_id = 0 if str(device) == 'cuda' else -1

## Data Pre-Processing

In [3]:
from google.colab import drive
drive.mount('/content/drive')
# folder = pd.read_csv("/content/drive/Shared With Me/folder_0/folder_1/file.csv")

Mounted at /content/drive


In [4]:
# Load data
path = "/content/drive/My Drive/0 GROUP PROJECTS/CIS 5300 Final Project/5300 Project Data/exp_data/data_email_short/"
# train_source = pd.read_csv(path + 'train_source.txt', sep = '\n')
source = []
s = open(path + 'train_source.txt', 'r', encoding="utf-8")
for line in s:
  source.append(line.strip())

target = []
t = open(path + 'train_target.txt', 'r', encoding="utf-8")
for line in t:
  target.append(line.strip())

train = pd.DataFrame({"Source" : source, "Target" : target})
train

,Source,Target
0,"Subject: ep strategy|||Andy: I'm fried, frazzl...",Andy expresses stress about his current worklo...
1,"Subject: tibco layoffs, wireless cuts|||Steve:...",There is going to be lay offs and they are try...
2,Subject: are the knicks playing tonight?|||Dou...,Doug makes a joke about the Knicks. Fernand sa...
3,"Subject: func test failures|||Ravi: Sharon, I ...",Ravi tells Sharon there are some functional te...
4,"Subject: feedback from market|||Peter: Ron, I ...",The AvocadoIT team discuss problems with mobil...
...,...,...
1795,Subject: agenda for program mgmt meeting tbh 1...,Hideki cannot make the meeting and instead has...
1796,"Subject: cage storage|||Mark: Debbie, I need y...",Mark needs Debbie's help with items inside the...
1797,Subject: hp & exodus emobility initiative|||Ro...,Ron writes to the team regarding the HP & Exod...
1798,Subject: devices that we support today|||Rober...,Robert makes a list for Dan of the devices the...


In [5]:
# Load data
source = []
s = open(path + 'test_source.txt', 'r', encoding="utf-8")
for line in s:
  source.append(line.strip())

target = []
t = open(path + 'test_target.txt', 'r', encoding="utf-8")
for line in t:
  target.append(line.strip())

test = pd.DataFrame({"Source" : source, "Target" : target})
test

,Source,Target
0,"Subject: 1:30pm conference call|||Ty: Dan, Can...",Ty asks Dan if he can take a conference call w...
1,Subject: 12/20/00 need flip chart for sausalit...,Jackie needs flip chart for sausalito conf rm ...
2,Subject: environment for respond|||Sujan: Guys...,Sujan reminds everyone they need a respond env...
3,"Subject: ert user guide|||Ruth: Piyush, I know...",Ruth requests Piyush check the status of a gui...
4,Subject: 4.5 patch for japan|||Divakar: Rajeev...,Diva tells Rajeev Japan needs a patch. Rajeev ...
...,...,...
495,Subject: usps demo....tuesday demo!|||Ray: I ...,Ray communicates the demo date for USPS and as...
496,Subject: training for accenture developers in ...,Carlos asks for contact information to arrange...
497,Subject: ui postmortem minutes|||Mark: Here is...,Mark sends a summary of a recent event to the ...
498,Subject: system admin training in ep japan|||R...,There is some confusion about who will teach a...


In [6]:
# Load data
source = []
s = open(path + 'dev_source.txt', 'r', encoding="utf-8")
for line in s:
  source.append(line.strip())

target = []
t = open(path + 'dev_target.txt', 'r', encoding="utf-8")
for line in t:
  target.append(line.strip())

dev = pd.DataFrame({"Source" : source, "Target" : target})
dev

,Source,Target
0,Subject: bug 2664|||Sharon: Hi Ravi - this is ...,Sharon tells Ravi about an invalid link. Ravi ...
1,Subject: thanks for the hospitality and courte...,Dave thanks the group for their hospitality on...
2,Subject: testservlet update for sp4|||Howard: ...,Howard provides an action plan. Rajdeep descri...
3,Subject: andy at javaone on monday|||Andy: I'l...,Andy is at JavaOne and asks Jamie to review th...
4,Subject: avocadoit/forrester follow-up|||Ryan:...,Agenda for meeting is presented by Amanda and ...
...,...,...
244,Subject: avocadoit booth staff needed at wirel...,Debbie asks for volunteers. Laura volunteers. ...
245,Subject: marketing manager job description|||M...,Marcia writes to the team about the pending ma...
246,Subject: alaska bug|||Kavitha: This bug has be...,Kavitha notes that a bug has been fixed and as...
247,Subject: offline admin guide 4.0 - review|||De...,AvocadoIT completes an offline administration ...


In [7]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/models/t5/tokenization_t5.py:240: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the

In [8]:
prefix = 'summarize: '
def add_summarize_prompt(df):
  inputs = [prefix + doc for doc in df["Source"]]
  df['Source'] = inputs
  model_inputs = tokenizer(inputs, max_length = 512, truncation = True)
  labels = tokenizer(text_target=df["Target"], max_length = 32, truncation = True)
  model_inputs["labels"] = labels["input_ids"]
  return model_inputs

ds_train = Dataset.from_pandas(train)
tokenized_train = ds_train.map(add_summarize_prompt, batched = True)

ds_dev = Dataset.from_pandas(dev)
tokenized_dev = ds_dev.map(add_summarize_prompt, batched = True)

ds_test = Dataset.from_pandas(test)
tokenized_test = ds_test.map(add_summarize_prompt, batched = True)

Map:   0%|          | 0/1800 [00:00<?, ? examples/s]

Map:   0%|          | 0/249 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

## Strong Baseline: Fine-Tuned T5

In [9]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [10]:
!git clone https://github.com/ThomasScialom/summa-qa.git

Cloning into 'summa-qa'...
remote: Enumerating objects: 57, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 57 (delta 0), reused 0 (delta 0), pack-reused 54
Receiving objects: 100% (57/57), 132.15 KiB | 2.13 MiB/s, done.
Resolving deltas: 100% (23/23), done.


In [38]:
!apt install python3.10-venv
!python -m venv .summaqa
!.summaqa/bin/pip install torch
!.summaqa/bin/pip install -U spacy==2.3.8
!.summaqa/bin/pip install -U srsly==1.0.7
!.summaqa/bin/pip install -U catalogue==1.0.2
!.summaqa/bin/pip install -U scikit-learn
!.summaqa/bin/pip install -e summa-qa/

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3.10-venv is already the newest version (3.10.12-1~22.04.3).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
Obtaining file:///content/summa-qa
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Attempting uninstall: summaqa
    Found existing installation: summaqa 0.1.0
    Uninstalling summaqa-0.1.0:
      Successfully uninstalled summaqa-0.1.0
  Running setup.py develop for summaqa


In [39]:
def compute_metrics(eval_pred):
  predictions, labels = eval_pred
  decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
  labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
  decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

  result = rouge.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)

  prediction_lens = [np.count_nonzero(pred != tokenizer.pad_token_id) for pred in predictions]
  result["gen_len"] = np.mean(prediction_lens)

  return {k: round(v, 4) for k, v in result.items()}

In [40]:
# ft_model = AutoModelForSeq2SeqLM.from_pretrained('t5-base')
ft_model = T5ForConditionalGeneration.from_pretrained('t5-base').to('cuda')

In [17]:
checkpoint = "t5-base"
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=checkpoint)
rouge = load("rouge")
bertscore = load("bertscore")

In [52]:
# Train model
training_args = Seq2SeqTrainingArguments(
    output_dir="ft_t5_model",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    weight_decay=0.01,
    save_total_limit=3,
    num_train_epochs=4,
    predict_with_generate=True,
    fp16=True,
)

trainer = Seq2SeqTrainer(
    model=ft_model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_dev,
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum,Gen Len
1,No log,2.272419,0.339100,0.110100,0.290100,0.290100,18.871500
2,2.608600,2.184788,0.341900,0.109100,0.292900,0.293000,18.983900
3,2.275000,2.154949,0.344800,0.112600,0.294100,0.294200,18.963900
4,2.176000,2.149062,0.340800,0.109700,0.291400,0.292000,18.963900


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1355: UserWarning: Using the model-agnostic default `max_length` (=20) to control

TrainOutput(global_step=1800, training_loss=2.317943115234375, metrics={'train_runtime': 812.7292, 'train_samples_per_second': 8.859, 'train_steps_per_second': 2.215, 'total_flos': 3693091732070400.0, 'train_loss': 2.317943115234375, 'epoch': 4.0})

In [53]:
trainer.save_model('./ft_t5_model_summaqa_sup')
ft_summarizer = pipeline("summarization", model="/content/ft_t5_model_summaqa_sup")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Fine tune conditional generation model to use GPUs

In [54]:
# Assuming 'test' is your DataFrame and 'Source' is the column with text to summarize

# Load the model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('./ft_t5_model_summaqa_sup').to('cuda')

def batch_summarize(texts, batch_size=4):
    summaries = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]

        # Tokenize the texts and prepare the input tensor
        inputs = tokenizer(batch_texts, padding=True, truncation=True, max_length=512, return_tensors='pt').to('cuda')

        # Generate summaries
        with torch.no_grad():
            summary_ids = model.generate(inputs['input_ids'], num_beams=4, max_length=50, early_stopping=True)

        # Decode and add to the list
        batch_summaries = [tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]
        summaries.extend(batch_summaries)

    return summaries

# Apply batch summarization
test['Predicted'] = batch_summarize(tokenized_test['Source'])

## Model Evaluation & Example Summaries

In [49]:
import json
import subprocess

def evaluate_corpus(labels, preds):
  l = open('/content/labels.txt', 'w', encoding="utf-8")
  l.write(json.dumps(labels))
  l.close()
  p = open('/content/preds.txt', 'w', encoding="utf-8")
  p.write(json.dumps(preds))
  p.close()
  result = subprocess.run(['/content/.summaqa/bin/python', '/content/evaluate_corpus.py'], stdout=subprocess.PIPE).stdout.decode('utf-8')
  return result

In [50]:
# Token indices sequence length is longer than the specified maximum sequence length for this model (645 > 512). Running this sequence through the model will result in indexing errors
#ft_rouge_results = rouge.compute(predictions = test['Predicted'], references = test['Target'])
#ft_berts_results = bertscore.compute(predictions = test['Predicted'], references = test['Target'], lang = 'en')
ft_summaqa_sup_results = evaluate_corpus(test['Target'].to_list(), test['Predicted'].to_list())
#ft_summaqa_unsup_model_results = evaluate_corpus(test['Source'].to_list(), test['Predicted'].to_list())
#ft_summaqa_unsup_human_results = evaluate_corpus(test['Source'].to_list(), test['Target'].to_list())

In [30]:
ft_rouge_results

{'rouge1': 0.31835172041107784,
 'rouge2': 0.09882060832311235,
 'rougeL': 0.26257003723778816,
 'rougeLsum': 0.2622093522356507}

In [31]:
ft_berts_results

{'precision': [0.885514497756958,
  0.9103781580924988,
  0.9185308814048767,
  0.8848789930343628,
  0.8484417200088501,
  0.8626782894134521,
  0.8510138988494873,
  0.8770890235900879,
  0.8535192608833313,
  0.9361916184425354,
  0.81995689868927,
  0.8789781332015991,
  0.8515605926513672,
  0.9141538739204407,
  0.8869668245315552,
  0.9059656262397766,
  0.8872361183166504,
  0.9037200212478638,
  0.8592966198921204,
  0.8669044971466064,
  0.8777458667755127,
  0.8578460216522217,
  0.8782757520675659,
  0.8577117323875427,
  0.9329012036323547,
  0.8732113838195801,
  0.8776817917823792,
  0.9012840390205383,
  0.8650364875793457,
  0.8717403411865234,
  0.8692576289176941,
  0.9153975248336792,
  0.8863510489463806,
  0.8738831281661987,
  0.9161757230758667,
  0.9042315483093262,
  0.8767898082733154,
  0.862923264503479,
  0.8430113792419434,
  0.8867502212524414,
  0.9383583664894104,
  0.9032983183860779,
  0.8855341076850891,
  0.9109084010124207,
  0.9207214117050171,
 

In [32]:
avg_bert_precision = np.mean(ft_berts_results["precision"])
avg_bert_accuracy = np.mean(ft_berts_results["recall"])
avg_bert_f1 = np.mean(ft_berts_results["f1"])
print("Precision:", avg_bert_precision)
print("Recall:", avg_bert_accuracy)
print("F1-Score", avg_bert_f1)

Precision: 0.8842022165060043
Recall: 0.8842543023824692
F1-Score 0.8840908049345017


In [51]:
ft_summaqa_sup_results

''

In [ ]:
ft_summaqa_unsup_model_results

'{"avg_prob": 0.14506175279360667, "avg_fscore": 0.08025748243554728}\n'

In [ ]:
ft_summaqa_unsup_human_results

'{"avg_prob": 0.11138253964919863, "avg_fscore": 0.06648447417537756}\n'

In [ ]:
# Example 1
print(test['Predicted'][12])
print(tokenized_test['Source'][12])
print(test['Target'][12])

Susan asks for a Wednesday lunch to celebrate the one year anniversary of Michelle & David and Mark's birthday. David says he will skip it. Tamra
summarize: Subject: lunch this week|||Susan: All, Regarding our lunch this week to celebrate the one year anniversaries for Michelle & David, and Mark's birthday, I have a request to make it Wednesday instead of Tuesday.  Does anyone have an objection to this? Susan|||David: I have another lunch engagement Wed, but I will skip it if everyone else wants to move our lunch. David|||Tamra: Susan, Wednesday works out better for me as well.  I have a doctor's appointment tomorrow during lunch. Tamra
Susan emails everyone about an anniversary and offers to change the date. David says he is busy but is willing to go with the majority.  Tamra agrees with Susan's date.


In [ ]:
# Example 2
print(test['Predicted'][1])
print(tokenized_test['Source'][1])
print(test['Target'][1])

Jackie asks for a flip chart. Yolanda asks for a flip chart.
summarize: Subject: 12/20/00 need flip chart for sausalito conf rm.|||Jackie: Also, Please clean out the fridge. Thank you. Jackie|||Yolanda: are you going to need flip chart today? and yes, i will clean the refrigerator this morning, remember, i asked you to let me know when the meeting was over the other day, so i could clean the refrigerator.|||Jackie: Thanks Yoli!! Sorry, I couldn't remember if I had sent this one or not. I appreciate your efforts and hard work to help me out!!! :)|||Yolanda: you have sent me this message 3 times already and iam currently looking for one|||Jackie: Sorry!!! Thanks again.
Jackie needs flip chart for sausalito conf rm and fridge cleaned. Yolanda verifies she'll clean the fridge this morning.


In [ ]:
# Example 3
print(test['Predicted'][2])
print(tokenized_test['Source'][2])
print(test['Target'][2])

Sujan asks for a Respond environment.
summarize: Subject: environment for respond|||Sujan: Guys, We need a Respond environment. For that I need to order a "standard destop" hardware configuration which we can use as the server. As it will only be used as a server, we don't need the fancy monitor (ie 15" will be ok). Please order one for my group. Let me know the availability and if you need more specific details. Thanks, Sujan|||Sujan: Guys, We need a Respond environment. For that I need to order a "standard destop" hardware configuration which we can use as the server. As it will only be used as a server, we don't need the fancy monitor (ie 15" will be ok). Please order one for my group. Let me know the availability and if you need more specific details. Thanks, Sujan|||Marek: Sujan, I'm not sure if you already took care of that, but the external IP is very important (if we want to test Portal from a real phone), and depending on Sitraka's setup, might turn out to be harder to get tha

In [ ]:
test.to_csv('/content/drive/My Drive/t5_ft_data.csv')